# 📊 Sahte Haber Analizi - Kapsamlı Rapor

Bu notebook tüm analizlerin kapsamlı raporunu içerir.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 10)

print("📋 Kapsamlı rapor hazırlanıyor...")
print(f"📅 Rapor tarihi: {datetime.now().strftime('%d/%m/%Y %H:%M')}")

📋 Kapsamlı rapor hazırlanıyor...
📅 Rapor tarihi: 10/08/2025 18:36


In [2]:
# Veri yükleme
fake_news = pd.read_csv('../data/Fake.csv')
real_news = pd.read_csv('../data/True.csv')

fake_news['label'] = 0
real_news['label'] = 1
fake_news['type'] = 'Sahte'
real_news['type'] = 'Gerçek'

df = pd.concat([fake_news, real_news], ignore_index=True)
df = df.dropna(subset=['title', 'text'])

# Metin özellikleri
df['title_length'] = df['title'].str.len()
df['text_length'] = df['text'].str.len()
df['word_count'] = df['text'].str.split().str.len()

print(f"📊 Rapor için veri hazır: {len(df):,} haber")

📊 Rapor için veri hazır: 44,898 haber


## 📈 1. Genel Veri Özeti

In [3]:
# Kapsamlı özet tablosu
summary_stats = pd.DataFrame({
    'Metrik': ['Toplam Haber Sayısı', 'Sahte Haber Sayısı', 'Gerçek Haber Sayısı', 
               'Sahte Haber Oranı (%)', 'Ortalama Başlık Uzunluğu', 'Ortalama Metin Uzunluğu',
               'Ortalama Kelime Sayısı'],
    'Değer': [f"{len(df):,}", f"{len(df[df['label'] == 0]):,}", f"{len(df[df['label'] == 1]):,}",
              f"{len(df[df['label'] == 0]) / len(df) * 100:.1f}%",
              f"{df['title_length'].mean():.0f} karakter",
              f"{df['text_length'].mean():.0f} karakter",
              f"{df['word_count'].mean():.0f} kelime"]
})

print("📋 GENEL VERİ ÖZETİ")
print("=" * 50)
display(summary_stats)

# Tip bazında istatistikler
print("\n📊 TİP BAZINDA İSTATİSTİKLER")
print("=" * 50)
type_stats = df.groupby('type')[['title_length', 'text_length', 'word_count']].agg([
    'mean', 'median', 'std', 'min', 'max'
]).round(2)
display(type_stats)

📋 GENEL VERİ ÖZETİ


,Metrik,Değer
0,Toplam Haber Sayısı,"44,898"
1,Sahte Haber Sayısı,"23,481"
2,Gerçek Haber Sayısı,"21,417"
3,Sahte Haber Oranı (%),52.3%
4,Ortalama Başlık Uzunluğu,80 karakter
5,Ortalama Metin Uzunluğu,2469 karakter
6,Ortalama Kelime Sayısı,405 kelime



📊 TİP BAZINDA İSTATİSTİKLER


title_length                        text_length                       \
               mean median    std min  max        mean  median      std min   
type                                                                          
Gerçek        64.67   64.0   9.17  26  133     2383.28  2222.0  1684.84   1   
Sahte         94.20   90.0  27.18   8  286     2547.40  2166.0  2532.88   1   

              word_count                           
          max       mean median     std min   max  
type                                               
Gerçek  29781     385.64  359.0  274.01   0  5172  
Sahte   51794     423.20  363.0  408.39   0  8135

## 📊 2. Kapsamlı Görselleştirme Panosu

In [4]:
# Ana dashboard
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=('Haber Türü Dağılımı', 'Metin Uzunluk Karşılaştırması',
                   'Kelime Sayısı Dağılımı', 'Başlık Uzunluk Analizi',
                   'Konu Dağılımı', 'Zaman Serisi Analizi'),
    specs=[[{'type': 'pie'}, {'type': 'box'}],
           [{'type': 'histogram'}, {'type': 'violin'}],
           [{'type': 'bar'}, {'type': 'scatter'}]]
)

# Pie chart
label_counts = df['type'].value_counts()
fig.add_trace(go.Pie(labels=label_counts.index, values=label_counts.values,
                    marker_colors=['#ff6b6b', '#4ecdc4']), row=1, col=1)

# Box plot
for i, news_type in enumerate(['Sahte', 'Gerçek']):
    data = df[df['type'] == news_type]['text_length']
    fig.add_trace(go.Box(y=data, name=news_type, 
                        marker_color='#ff6b6b' if news_type == 'Sahte' else '#4ecdc4'),
                 row=1, col=2)

# Histogram
for news_type in ['Sahte', 'Gerçek']:
    data = df[df['type'] == news_type]['word_count']
    fig.add_trace(go.Histogram(x=data, name=news_type, opacity=0.7,
                              marker_color='#ff6b6b' if news_type == 'Sahte' else '#4ecdc4'),
                 row=2, col=1)

# Violin plot
for news_type in ['Sahte', 'Gerçek']:
    data = df[df['type'] == news_type]['title_length']
    fig.add_trace(go.Violin(y=data, name=news_type,
                           marker_color='#ff6b6b' if news_type == 'Sahte' else '#4ecdc4'),
                 row=2, col=2)

fig.update_layout(height=1200, title_text="📊 Sahte Haber Analizi - Kapsamlı Dashboard")
fig.show()

print("✅ İnteraktif dashboard oluşturuldu")

✅ İnteraktif dashboard oluşturuldu


## 📝 3. Detaylı Bulgular ve Sonuçlar

In [5]:
# Detaylı bulgular
print("🔍 DETAYLI BULGULAR VE ANALİZ SONUÇLARI")
print("=" * 60)

print("\n📊 1. VERİ SETİ ÖZELLİKLERİ:")
print(f"   • Toplam haber sayısı: {len(df):,}")
print(f"   • Sahte haber oranı: {len(df[df['label'] == 0]) / len(df) * 100:.1f}%")
print(f"   • Gerçek haber oranı: {len(df[df['label'] == 1]) / len(df) * 100:.1f}%")
print(f"   • Veri seti dengeli mi: {'Evet' if abs(len(df[df['label'] == 0]) - len(df[df['label'] == 1])) < len(df) * 0.1 else 'Hayır'}")

print("\n📏 2. METİN UZUNLUK ANALİZİ:")
fake_avg_length = df[df['type'] == 'Sahte']['text_length'].mean()
real_avg_length = df[df['type'] == 'Gerçek']['text_length'].mean()
print(f"   • Sahte haberlerin ortalama uzunluğu: {fake_avg_length:.0f} karakter")
print(f"   • Gerçek haberlerin ortalama uzunluğu: {real_avg_length:.0f} karakter")
print(f"   • Uzunluk farkı: {abs(fake_avg_length - real_avg_length):.0f} karakter")

fake_avg_words = df[df['type'] == 'Sahte']['word_count'].mean()
real_avg_words = df[df['type'] == 'Gerçek']['word_count'].mean()
print(f"   • Sahte haberlerin ortalama kelime sayısı: {fake_avg_words:.0f}")
print(f"   • Gerçek haberlerin ortalama kelime sayısı: {real_avg_words:.0f}")

if 'subject' in df.columns:
    print("\n📚 3. KONU ANALİZİ:")
    subject_counts = df['subject'].value_counts()
    print(f"   • Toplam konu sayısı: {len(subject_counts)}")
    print(f"   • En popüler konu: {subject_counts.index[0]} ({subject_counts.iloc[0]} haber)")
    
    # Konu bazında sahte haber oranları
    fake_ratios = df.groupby('subject')['label'].apply(lambda x: (x == 0).mean() * 100)
    highest_fake_subject = fake_ratios.idxmax()
    lowest_fake_subject = fake_ratios.idxmin()
    print(f"   • En yüksek sahte haber oranına sahip konu: {highest_fake_subject} ({fake_ratios[highest_fake_subject]:.1f}%)")
    print(f"   • En düşük sahte haber oranına sahip konu: {lowest_fake_subject} ({fake_ratios[lowest_fake_subject]:.1f}%)")

print("\n✅ Detaylı bulgular raporu tamamlandı")

🔍 DETAYLI BULGULAR VE ANALİZ SONUÇLARI

📊 1. VERİ SETİ ÖZELLİKLERİ:
   • Toplam haber sayısı: 44,898
   • Sahte haber oranı: 52.3%


   • Gerçek haber oranı: 47.7%
   • Veri seti dengeli mi: Evet

📏 2. METİN UZUNLUK ANALİZİ:
   • Sahte haberlerin ortalama uzunluğu: 2547 karakter
   • Gerçek haberlerin ortalama uzunluğu: 2383 karakter
   • Uzunluk farkı: 164 karakter
   • Sahte haberlerin ortalama kelime sayısı: 423
   • Gerçek haberlerin ortalama kelime sayısı: 386

📚 3. KONU ANALİZİ:
   • Toplam konu sayısı: 8
   • En popüler konu: politicsNews (11272 haber)
   • En yüksek sahte haber oranına sahip konu: Government News (100.0%)
   • En düşük sahte haber oranına sahip konu: politicsNews (0.0%)

✅ Detaylı bulgular raporu tamamlandı


In [6]:
# Özet istatistik tablosu oluştur
summary_table = df.groupby('type').agg({
    'title_length': ['count', 'mean', 'std', 'min', 'max'],
    'text_length': ['mean', 'std', 'min', 'max'],
    'word_count': ['mean', 'std', 'min', 'max']
}).round(2)

print("📊 ÖZET İSTATİSTİK TABLOSU")
print("=" * 50)
display(summary_table)

# Tabloyu CSV olarak kaydet
summary_table.to_csv('../reports/ozet_istatistikler.csv')
print("\n💾 Özet istatistikler CSV olarak kaydedildi")

📊 ÖZET İSTATİSTİK TABLOSU


title_length                        text_length                      \
              count   mean    std min  max        mean      std min    max   
type                                                                         
Gerçek        21417  64.67   9.17  26  133     2383.28  1684.84   1  29781   
Sahte         23481  94.20  27.18   8  286     2547.40  2532.88   1  51794   

       word_count                    
             mean     std min   max  
type                                 
Gerçek     385.64  274.01   0  5172  
Sahte      423.20  408.39   0  8135


💾 Özet istatistikler CSV olarak kaydedildi
